In [0]:
%run "./config/v4/configurations-bw-kv-json"

In [0]:
%run "./config/v4/common"

data_source_id,schema_version,schema_ddl,logic_mapping
0.1.0,0.1.1,"`Line` STRING, `Station` STRING, `Part Number` STRING, `Database Code` STRING, `Serial Number` STRING, `Time` STRING, `Gun` STRING, `Job` STRING, `Pass` STRING, `Torque` STRING, `Ang (deg.)` STRING","{""line"":[""Line""], ""station_config"":[""Line"", ""Station""], ""sensor_config"":[""Line"", ""Station"", ""Gun"", ""Job""], ""part_number"": [""Part_Number""], ""serial_number"": [""Serial_Number""], ""measurement"":[""Torque"", ""Ang__deg__""], ""measured_time"":""Time""}"


data_source_id,client,location,line,source_type,folder_location
0.1.0,Borg Warner,XYZ,L14,csv,server://path


In [0]:
#**********************************************************************************************************************
# RESET STREAMING (for testing and dev only)
# use this code block to clear streaming checkpoints and output folders 
#**********************************************************************************************************************

dbutils.fs.rm(checkpointPath, True)
dbutils.fs.rm(pivotRootDir, True)
dbutils.fs.rm(cubeRunRootDir, True)


#**********************************************************************************************************************
# RESET PIVOT SCHEMA TABLE (for testing and dev only)
# use this code block to create an empty pivot schema table 
#**********************************************************************************************************************

pivot_schema_df = spark.createDataFrame(data=[],schema=T._parse_datatype_string(pivot_schema_table_ddl))
pivot_schema_df.registerTempTable("pivot_schema_table")
pivot_schema_df.show()

+------------+-------------+------------------+-------------+-----------+----------+
dataSourceId|schemaVersion|pivotSchemaVersion|pivotIndexDdl|featureList|updated_on|
+------------+-------------+------------------+-------------+-----------+----------+
+------------+-------------+------------------+-------------+-----------+----------+

In [0]:
import pyspark.sql.functions as f
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark import Row
from delta.tables import DeltaTable
from datetime import datetime
from pyspark.sql.types import *
import json

#**********************************************************************************************************************
# HOUSE KEEPING
#**********************************************************************************************************************

# set the basic spark configurations
spark.conf.set("spark.sql.shuffle.partitions", 8)
spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled", True)

# load configuration and other variables through Notebook workflow
#dbutils.notebook.run("./configurations", 60)


#**********************************************************************************************************************
# DATA INGESTION: read stream from Kafka topic and apply ingress data schema
#**********************************************************************************************************************

input_df = (
  spark
  .readStream
  .format("kafka")
  .option("kafka.bootstrap.servers", conf['bootstrap.servers'])
  .option("kafka.security.protocol", "SASL_SSL")
  .option("kafka.sasl.jaas.config", "kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username='{}' password='{}';".format(conf['sasl.username'], conf['sasl.password']))
  .option("kafka.ssl.endpoint.identification.algorithm", "https")
  .option("kafka.sasl.mechanism", "PLAIN")
  .option("subscribe", conf['topic'])
  .option("startingOffsets", conf['startingOffsets'])
  .option("failOnDataLoss", "false")
  .option("maxOffsetsPerTrigger", "1")
  .load()
  .select(col("value").cast("STRING"), col("key").cast("STRING")) 
  .select(from_json("value", ingress_schema).alias("ingress_json"))
)


#**********************************************************************************************************************
# TRANSFORM STEP 1/5: extract data source id, schema version, and measurement dataframe from ingested Kafka record.
# The column headers of the measurement dataframe are cleaned to remove illegal characters.
#**********************************************************************************************************************

def replace_special_char(column_name):
  """ replace all non-alphanumeric characters with an underscore. """
  return regexp_replace(column_name, r'[^a-zA-Z0-9]', '_')

formatted_df = (input_df
                .select(col("ingress_json.dataSourceId").alias("dataSourceId"), 
                        col("ingress_json.schemaVersion").alias("schemaVersion"), explode("ingress_json.rows").alias("records"))
                .withColumn("headers", transform(f.map_keys("records"), replace_special_char))
                .withColumn("data", map_values("records"))
               .withColumn("mapped_fields",  map_from_arrays("headers", "data"))
               .select("dataSourceId", "schemaVersion", "mapped_fields")
              )


#**********************************************************************************************************************
# TRANSFORM STEP 2/5: apply business logic mapping to incoming data columns
# NOTE: the business-mapping config json string is read from the business-schema table and applied in realtime to
# the data records. Records with different data source id and schema version will have different schemas applied.
#**********************************************************************************************************************

def mapLogicalFields(fields, logic_mapping):
  """
  udf to map data fields according to business logic mapping config
  
  Parameters
    ----------
    fields : array<map>
        An array of maps with column header as keys and data as values. One array corresponds to one row of data 
    logic_mapping : string
        The json string containing business mapping config

    Returns
    -------
    logic_mapped_dict : dict
        A dict with the logic entites (line, station, sensor, part number, serial number, timestamp) as keys
        and data mapped from the incoming data records as values
  """
  
  mappings = json.loads(logic_mapping)
  ret, measurement_dict = {}, {}
  for attr in mappings:
    if attr=='measured_time':
      ret[config_col_mapping[attr]]=fields[mappings[attr]]
    elif attr=='measurement':
      for m in mappings[attr]:
        measurement_dict[m]=fields[m]
    else:
      ret[config_col_mapping[attr]] = '_'.join([fields[c] for c in mappings[attr]])
  ret['measurements']=measurement_dict
  return ret

# register the udf
logicMapUDF = udf(lambda x, y: mapLogicalFields(x, y), mapped_schema)


mapped_df = (formatted_df
             .join(business_schema_df, 
                   (formatted_df["dataSourceId"]==business_schema_df["data_source_id"]) & 
                   (formatted_df["schemaVersion"]==business_schema_df["schema_version"])    # EXCEPTION HANDLING for unknown source/schemaVersion
                  )
             .withColumn("logic_mapped", logicMapUDF(col("mapped_fields"), col("logic_mapping")))
             .select("dataSourceId", "schemaVersion", "logic_mapped")
             .filter("logic_mapped.serial_number!=''")     # DISCUSSION: how to handle records with missing serial numbers
            )


#**********************************************************************************************************************
# TRANSFORM STEP 3/5: replace sensor name with uuid, then return a dataframe with business entities as column headers, 
# a list of feature names, and a kev-value list of feature-name and data
#**********************************************************************************************************************
uuid_df = (mapped_df
           .join(sensorDf.select("sensor", "sensor_uuid"), mapped_df["logic_mapped.sensor"]==sensorDf["sensor"], "left")
           .withColumn("feature_names", transform(map_keys("logic_mapped.measurements"), lambda x: concat(col('sensor_uuid'), lit('_'), x)))
           .withColumn("feature_json", transform(arrays_zip("feature_names", map_values("logic_mapped.measurements")),
                                                 lambda x: concat(lit('"'), x["feature_names"], lit('":"'), x['1'], lit('"'))
                                                ))
           .select("dataSourceId", "schemaVersion",
                   col("logic_mapped.line").alias('line'), col("logic_mapped.part_number").alias('part_number'), 
                   col("logic_mapped.serial_number").alias('serial_number'), col("logic_mapped.station").alias('station'), 
                   col("logic_mapped.timestamp").alias('timestamp'), "feature_json", "feature_names"
                  )
          )


#**********************************************************************************************************************
# TRANSFORM STEP 4/5: collapse rows with same timestamp for each serial number into a single row, and create a
# 'measurement_date' column to be used as partition column
#**********************************************************************************************************************
collapsed_df = (uuid_df
                 .groupBy("dataSourceId", "schemaVersion", "line", "part_number","serial_number", "timestamp")
                 .agg(collect_set('station').alias("station_list"), 
                      concat(lit("{"), concat_ws(", ", array_distinct(flatten(collect_list("feature_json")))), lit("}")).alias("feature_json"),
                      array_distinct(flatten(collect_list("feature_names"))).alias("feature_names")
                     )
                 .withColumn("measurement_date", to_date("timestamp"))
                )


#**********************************************************************************************************************
# TRANSFORM STEP 5/5: join with source_meta_data_table to get full name of the line, which will decide which cube
# the transformed data is dropped into
#**********************************************************************************************************************
pivot_prep_df = (collapsed_df
                 .join(
                   data_source_meta_df.select("data_source_id", concat_ws("_", "client", "location", "line").alias("line_full_name")), 
                   collapsed_df["dataSourceId"]==data_source_meta_df["data_source_id"])
                 .withColumn("line_full_name", replace_special_char("line_full_name"))
                 .drop("data_source_id")
                )


display(pivot_prep_df)

dataSourceId,schemaVersion,line,part_number,serial_number,timestamp,station_list,feature_json,feature_names,measurement_date,line_full_name
0.1.0,0.1.1,L14_Core,12639700098,0882112380500070,2021-04-12 05:15:54.950,List(L14_Core_03),"{""a65c94ea-4616-4ece-ac87-d919cc97d035_Torque"":""155.85"", ""a65c94ea-4616-4ece-ac87-d919cc97d035_Ang__deg__"":""90""}","List(a65c94ea-4616-4ece-ac87-d919cc97d035_Torque, a65c94ea-4616-4ece-ac87-d919cc97d035_Ang__deg__)",2021-04-12,Borg_Warner_XYZ_L14
0.1.0,0.1.1,L14_Final,12639700098,0902114333500078,2021-04-12 16:26:52.337,List(L14_Final_04),"{""30963d81-db2e-467f-8a4c-28a37a15da8c_Torque"":""13.63"", ""30963d81-db2e-467f-8a4c-28a37a15da8c_Ang__deg__"":""64""}","List(30963d81-db2e-467f-8a4c-28a37a15da8c_Torque, 30963d81-db2e-467f-8a4c-28a37a15da8c_Ang__deg__)",2021-04-12,Borg_Warner_XYZ_L14
0.1.0,0.1.1,L14_Final,12639700098,0882111554700061,2021-04-12 05:17:09.563,List(L14_Final_04),"{""30963d81-db2e-467f-8a4c-28a37a15da8c_Torque"":""13.68"", ""30963d81-db2e-467f-8a4c-28a37a15da8c_Ang__deg__"":""59""}","List(30963d81-db2e-467f-8a4c-28a37a15da8c_Torque, 30963d81-db2e-467f-8a4c-28a37a15da8c_Ang__deg__)",2021-04-12,Borg_Warner_XYZ_L14
0.1.0,0.1.1,L14_Final,12769700033,0000338751200150,2021-04-09 20:50:38.590,List(L14_Final_03),"{""e6b8d50e-a042-4a6a-8d75-6519b3c02934_Torque"":""10.46"", ""e6b8d50e-a042-4a6a-8d75-6519b3c02934_Ang__deg__"":""43""}","List(e6b8d50e-a042-4a6a-8d75-6519b3c02934_Torque, e6b8d50e-a042-4a6a-8d75-6519b3c02934_Ang__deg__)",2021-04-09,Borg_Warner_XYZ_L14
0.1.0,0.1.1,L14_Final,12639700098,0482110035100049,2021-03-16 07:44:15.027,List(L14_Final_04),"{""30963d81-db2e-467f-8a4c-28a37a15da8c_Torque"":""13.67"", ""30963d81-db2e-467f-8a4c-28a37a15da8c_Ang__deg__"":""52""}","List(30963d81-db2e-467f-8a4c-28a37a15da8c_Torque, 30963d81-db2e-467f-8a4c-28a37a15da8c_Ang__deg__)",2021-03-16,Borg_Warner_XYZ_L14
0.1.0,0.1.1,L14_Final,12639700098,0842119511100133,2021-04-12 13:22:36.767,List(L14_Final_04),"{""f269a193-9198-434b-bc4c-fb46f7cdb1f3_Torque"":""17.02"", ""f269a193-9198-434b-bc4c-fb46f7cdb1f3_Ang__deg__"":""1319""}","List(f269a193-9198-434b-bc4c-fb46f7cdb1f3_Torque, f269a193-9198-434b-bc4c-fb46f7cdb1f3_Ang__deg__)",2021-04-12,Borg_Warner_XYZ_L14
0.1.0,0.1.1,L14_Core,12639700098,0542106071900004,2021-03-15 13:01:52.113,List(L14_Core_03),"{""a65c94ea-4616-4ece-ac87-d919cc97d035_Torque"":""165.63"", ""a65c94ea-4616-4ece-ac87-d919cc97d035_Ang__deg__"":""90""}","List(a65c94ea-4616-4ece-ac87-d919cc97d035_Torque, a65c94ea-4616-4ece-ac87-d919cc97d035_Ang__deg__)",2021-03-15,Borg_Warner_XYZ_L14
0.1.0,0.1.1,L14_Final,12769700033,0000338751200150,2021-03-26 00:55:39.450,List(L14_Final_04),"{""30963d81-db2e-467f-8a4c-28a37a15da8c_Torque"":""13.70"", ""30963d81-db2e-467f-8a4c-28a37a15da8c_Ang__deg__"":""54""}","List(30963d81-db2e-467f-8a4c-28a37a15da8c_Torque, 30963d81-db2e-467f-8a4c-28a37a15da8c_Ang__deg__)",2021-03-26,Borg_Warner_XYZ_L14
0.1.0,0.1.1,L14_Final,12639700098,14A077210099,2021-03-18 10:05:24.763,List(L14_Final_06),"{""4ad3b2c5-5198-4c11-ac65-86aef08f3db3_Torque"":""17.00"", ""4ad3b2c5-5198-4c11-ac65-86aef08f3db3_Ang__deg__"":""932""}","List(4ad3b2c5-5198-4c11-ac65-86aef08f3db3_Torque, 4ad3b2c5-5198-4c11-ac65-86aef08f3db3_Ang__deg__)",2021-03-18,Borg_Warner_XYZ_L14
0.1.0,0.1.1,L14_Final,12639700098,0542106411600017,2021-03-15 12:58:59.017,List(L14_Final_04),"{""30963d81-db2e-467f-8a4c-28a37a15da8c_Torque"":""13.65"", ""30963d81-db2e-467f-8a4c-28a37a15da8c_Ang__deg__"":""60""}","List(30963d81-db2e-467f-8a4c-28a37a15da8c_Torque, 30963d81-db2e-467f-8a4c-28a37a15da8c_Ang__deg__)",2021-03-15,Borg_Warner_XYZ_L14


In [0]:
### helper functions needed to dynamically generate and update pivot table schema

def getCurrPivotSchemaLocal(dataSourceId, schemaVersion):
  """
  returns the current features list from the pivot-schama-table and the computed next pivot schema version number 
  
  Parameters
    ----------
    dataSourceId : string
        The ID string of the data source 
    schemaVersion : string
        The data source schema version of the processed records

    Returns
    -------
    features_list : set
        A list of current features
    next_pivot_schema_version : int
        Version number of the next pivot schema to be inserted
  """
  
  res = spark.sql(f'select featureList, pivotSchemaVersion from pivot_schema_table \
                   where dataSourceId="{dataSourceId}" and schemaVersion="{schemaVersion}" and pivotSchemaVersion= \
                   (select max(pivotSchemaVersion) from pivot_schema_table where dataSourceId="{dataSourceId}" \
                   and schemaVersion="{schemaVersion}")').first()
  if res is None:
    return set(), 1
  return set(res[0].split(',')), res[1]+1


### TO BE IMPLEMENTED
def getCurrPivotSchemaPostgres(dataSourceId, schemaVersion):
  """ returns the current features list from a postgreSQL table and the computed next pivot schema version number """
  pass


def updatePivotSchemaLocal(data_source_id, schema_version, pivot_idx_ddl, pivot_schema_version, feature_list):
  """
  updates the features list and pivot-schema version in the pivot-schama-table 
  and returns the next pivot-schema version number 
  
  Parameters
    ----------
    data_source_id : string
        The ID string of the data source 
    schema_version : string
        The data source schema version of the processed records
    pivot_idx_ddl : string
        The ddl string of index columns in the pivot-schema-table
    pivot_schema_version : int
        Pivot-schema version number of the new schema to be inserted
    feature_list : set
        A list of current features in the new schema

    Returns
    -------
    next_pivot_schema_version : int
        Version number of the next pivot schema to be inserted
  """
  
  feature_list = ','.join(feature_list)
  pivot_schema_df = (spark.sql("select * from pivot_schema_table")
                   .union(spark.createDataFrame([(data_source_id, schema_version, pivot_schema_version, pivot_idx_ddl, 
                                                  feature_list, datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:23])]))
                  )
  pivot_schema_df.registerTempTable("pivot_schema_table")
  return pivot_schema_version+1


### TO BE IMPLEMENTED
def update_pivot_schema_postgres(dataSourceId, schemaVersion):
  """
  updates the features list and pivot-schema version in the postgreSQL table 
  and returns the next pivot-schema version number 
  """
  pass


def getDataSourceAndSchemaVersion(df):
  """
  returns the first data source id and schema version from a given dataframe 
  
  Parameters
    ----------
    df : dataframe
        A dataframe containing 'dataSourceId' and 'schemaVersion' columns

    Returns
    -------
    dataSourceId : string
        Data source ID of the data record
    SchemaVersion : int 
        Schema version of the data record
  """
  res = df.select("dataSourceId", "schemaVersion").first()
  if res is None:
    return "", ""
  return res[0], res[1]


def getFeatureListFromDF(df):
  """
  returns the list of unique feature names from a given dataframe 
  
  Parameters
    ----------
    df : dataframe
        A dataframe containing 'feature_names' column

    Returns
    -------
    features_list : list
        A list of unique feature names
  """
  
  res = df.agg(array_distinct(flatten(collect_list("feature_names")))).first()
  if res is None:
    return []
  return res[0]


def getFeatureListLocal(dataSourceId, schemaVersion):
  """
  returns the current features list from the pivot-schama-table 
  
  Parameters
    ----------
    dataSourceId : string
        The ID string of the data source 
    schemaVersion : string
        The schema version of the data source

    Returns
    -------
    features_list : set
        A list of current features
  """
  res = spark.sql(f'select featureList from pivot_schema_table \
                   where dataSourceId="{dataSourceId}" and schemaVersion="{schemaVersion}" and pivotSchemaVersion= \
                   (select max(pivotSchemaVersion) from pivot_schema_table where dataSourceId="{dataSourceId}" and \
                   schemaVersion="{schemaVersion}")').first()
  if res is None:
    return set()
  return set(res[0].split(','))


def initialize_delta_sink(line):
  """ create an empty delta table in the cube directory for a given line """
  (spark
 .createDataFrame([], schema=T._parse_datatype_string(pivot_idx_ddl))
 .write
 .option("mergeSchema", "true")
 .partitionBy("measurement_date") 
 .format("delta")
 .mode("append")
 .save(pivotRootDir + f"{line}/"))
  
  
def load_features_for_line(data_source_id, schema_version):
  """
  returns the current features list, next pivot schema version, and current features ddl string for
  given data_source_id and schema_version
  
  Parameters
    ----------
    dataSourceId : string
        The ID string of the data source 
    schemaVersion : string
        The schema version of the data source

    Returns
    -------
    curr_features_list : set
        A list of current features
    next_pivot_schema_version : int
        The next pivot schema version
    curr_feature_ddl : string
        A ddl string of the current features
  """
  curr_feature_list, next_pivot_schema_version = getCurrPivotSchemaLocal(data_source_id, schema_version)
  curr_feature_ddl = "`" + '` STRING, `'.join(sorted(curr_feature_list)) + "` STRING"
  return curr_feature_list, next_pivot_schema_version, curr_feature_ddl


def process_batch_for_line(line, line_df, data_source_id, schema_version):
  """
  Process batch dataframe for a given line, create a new version of pivot schema if new features are seen, 
  and updates the delta table sink by performing 'upserts' using ("serial_number", "part_number", 
  "timestamp", "station_list") as keys.
  
  Parameters
    ----------
    line : string
        Full name of the assembly line
    line_df : dataframe
        The batch dataframe for the given line
    data_source_id : string
        The ID string of the data source 
    schema_version : string
        The schema version of the data source
  """
  
  batch_feature_list = set(getFeatureListFromDF(line_df))
  
  if not batch_feature_list.issubset(lines_feature_dict[line]["curr_feature_list"]):
    lines_feature_dict[line]["prev_feature_list"] = lines_feature_dict[line]["curr_feature_list"]
    lines_feature_dict[line]["curr_feature_list"] = batch_feature_list.union(lines_feature_dict[line]["curr_feature_list"])
    lines_feature_dict[line]["next_pivot_schema_version"] = updatePivotSchemaLocal(data_source_id, 
                                                                                   schema_version, 
                                                                                   pivot_idx_ddl, 
                                                                                   lines_feature_dict[line]["next_pivot_schema_version"],
                                                                                   lines_feature_dict[line]["curr_feature_list"]
                                                                                  )
    lines_feature_dict[line]["curr_feature_ddl"] = "`" + '` STRING, `'.join(sorted(lines_feature_dict[line]["curr_feature_list"])) + "` STRING"
    
  # the target pivot table
  pivot_out = DeltaTable.forPath(spark, pivotRootDir + f"{line}/")

  # apply current schema to the batch dataframe
  schema_df = (line_df
            .withColumn("feature_json", from_json("feature_json", lines_feature_dict[line]["curr_feature_ddl"]))
            .select("serial_number", "part_number", "timestamp", "station_list", "measurement_date", col("feature_json.*"))
           )

  # rows in the pivot table needing to be updated
  if len(pivot_out.toDF().columns) > len(pivot_idx_ddl.split(',')):
    rows_to_update = (schema_df
                      .join(
                        pivot_out.toDF().alias("pivot_tab"), 
                        ["serial_number", "part_number", "timestamp", "station_list", "measurement_date"]
                      )
                      .select("serial_number", "part_number", "timestamp", "station_list", "measurement_date", "pivot_tab.*") 
                      .withColumn("feature_json", 
                                  concat(lit('{'), 
                                         concat_ws(',', 
                                                   *[concat(lit('"'), lit(c), lit('":"'), col(c), lit('"')) 
                                                     for c in lines_feature_dict[line]["prev_feature_list"]]
                                                  ),
                                         lit('}')
                                        )
                                 )
                      .withColumn("feature_json", from_json("feature_json", lines_feature_dict[line]["curr_feature_ddl"]))
                      .select("serial_number", "part_number", "timestamp", "station_list", "measurement_date", col("feature_json.*"))
                     )

    # prepare the updates dataframe
    updates_df = (schema_df
                 .union(rows_to_update)
                 .groupBy("serial_number", "part_number", "timestamp", "station_list", "measurement_date")
                 .agg(*[f.max(c).alias(c) for c in lines_feature_dict[line]["curr_feature_list"]])
                )
  else:
    updates_df = schema_df

  # upsert new data into Delta table
  match_on_key = """s.part_number=u.part_number and s.serial_number=u.serial_number and s.timestamp=u.timestamp and s.station_list=u.station_list""" 

  (pivot_out
   .alias("s")
   .merge(updates_df.alias("u"), match_on_key)
   .whenMatchedUpdateAll()
   .whenNotMatchedInsertAll()
   .execute()
  )

In [0]:
#**********************************************************************************************************************
# WRITE PIVOTTED RESULTS TO DELTA TABLE (THE CUBE)
# The pivot table is dynamically populated and updated with transformed streaming data
#**********************************************************************************************************************

lines_processed, lines_feature_dict = [], {}

def updateSchemaAndDelta(df, batch_id):
  """
  Process the batch dataframe. For each line in the dataframe, update the corresponding delta table.
  
  Parameters
    ----------
    df : dataframe
        The batch dataframe 
    batch_id : string
        The batch ID
  """
  global lines_processed, lines_feature_dict
  
  lines = df.select("line_full_name").distinct().collect()
  for a_row in lines:
    line = a_row["line_full_name"]
    line_df = df.filter(col("line_full_name")==line)
    data_source_id, schema_version =  getDataSourceAndSchemaVersion(line_df)   ## ADD HANDLER for line with multiple schema versions
    
    if not line in lines_processed:
      initialize_delta_sink(line)
      
      temp_dict = {}
      temp_dict["curr_feature_list"], \
      temp_dict["next_pivot_schema_version"], \
      temp_dict["curr_feature_ddl"] = load_features_for_line(data_source_id, schema_version)
      
      lines_feature_dict[line] = temp_dict
      
      lines_processed.append(line)
    
    process_batch_for_line(line, line_df, data_source_id, schema_version)


savePivot = (pivot_prep_df
             .writeStream
             .queryName("kafka_kv_to_pivot_delta_lake")
             .trigger(processingTime="1 seconds")
             .format("delta")
             .option("checkpointLocation", checkpointPath)
                 .option("mergeSchema", "true")
                 .foreachBatch(updateSchemaAndDelta)
             .outputMode("update")
             .start()
            )

#display(input_df)

In [0]:
display(spark.sql("select * from pivot_schema_table"))

dataSourceId,schemaVersion,pivotSchemaVersion,pivotIndexDdl,featureList,updated_on
0.1.0,0.1.1,1,"`part_number` STRING, `serial_number` STRING, `timestamp` STRING, `station_list` ARRAY, `measurement_date` DATE","b8d46933-5220-4bad-8221-c54715a835fd_Torque,30963d81-db2e-467f-8a4c-28a37a15da8c_Ang__deg__,e6b8d50e-a042-4a6a-8d75-6519b3c02934_Torque,a65c94ea-4616-4ece-ac87-d919cc97d035_Ang__deg__,e6b8d50e-a042-4a6a-8d75-6519b3c02934_Ang__deg__,a65c94ea-4616-4ece-ac87-d919cc97d035_Torque,b8d46933-5220-4bad-8221-c54715a835fd_Ang__deg__,f269a193-9198-434b-bc4c-fb46f7cdb1f3_Ang__deg__,30963d81-db2e-467f-8a4c-28a37a15da8c_Torque,f269a193-9198-434b-bc4c-fb46f7cdb1f3_Torque",2021-11-23 21:06:06.179
0.1.0,0.1.1,2,"`part_number` STRING, `serial_number` STRING, `timestamp` STRING, `station_list` ARRAY, `measurement_date` DATE","b8d46933-5220-4bad-8221-c54715a835fd_Torque,e6b8d50e-a042-4a6a-8d75-6519b3c02934_Torque,4ad3b2c5-5198-4c11-ac65-86aef08f3db3_Torque,a65c94ea-4616-4ece-ac87-d919cc97d035_Torque,e6b8d50e-a042-4a6a-8d75-6519b3c02934_Ang__deg__,30963d81-db2e-467f-8a4c-28a37a15da8c_Ang__deg__,a65c94ea-4616-4ece-ac87-d919cc97d035_Ang__deg__,30963d81-db2e-467f-8a4c-28a37a15da8c_Torque,b8d46933-5220-4bad-8221-c54715a835fd_Ang__deg__,f269a193-9198-434b-bc4c-fb46f7cdb1f3_Ang__deg__,4ad3b2c5-5198-4c11-ac65-86aef08f3db3_Ang__deg__,f269a193-9198-434b-bc4c-fb46f7cdb1f3_Torque",2021-11-23 21:06:13.186
0.1.0,0.1.1,3,"`part_number` STRING, `serial_number` STRING, `timestamp` STRING, `station_list` ARRAY, `measurement_date` DATE","b8d46933-5220-4bad-8221-c54715a835fd_Torque,e6b8d50e-a042-4a6a-8d75-6519b3c02934_Torque,4ad3b2c5-5198-4c11-ac65-86aef08f3db3_Torque,1f100034-a532-4a58-8b6c-f4444e074afe_Torque,1f100034-a532-4a58-8b6c-f4444e074afe_Ang__deg__,4ad3b2c5-5198-4c11-ac65-86aef08f3db3_Ang__deg__,a65c94ea-4616-4ece-ac87-d919cc97d035_Torque,e6b8d50e-a042-4a6a-8d75-6519b3c02934_Ang__deg__,30963d81-db2e-467f-8a4c-28a37a15da8c_Ang__deg__,a65c94ea-4616-4ece-ac87-d919cc97d035_Ang__deg__,b8d46933-5220-4bad-8221-c54715a835fd_Ang__deg__,f269a193-9198-434b-bc4c-fb46f7cdb1f3_Ang__deg__,30963d81-db2e-467f-8a4c-28a37a15da8c_Torque,f269a193-9198-434b-bc4c-fb46f7cdb1f3_Torque",2021-11-23 21:06:44.377


In [0]:
### For Dev and Testing, load back the table and view
pivot_load_df = spark.read.format('delta').load(pivotRootDir + "Borg_Warner_XYZ_L14/")
pivot_load_df.registerTempTable("pivot_results_table")
display(pivot_load_df)

part_number,serial_number,timestamp,station_list,measurement_date,30963d81-db2e-467f-8a4c-28a37a15da8c_Ang__deg__,30963d81-db2e-467f-8a4c-28a37a15da8c_Torque,a65c94ea-4616-4ece-ac87-d919cc97d035_Ang__deg__,a65c94ea-4616-4ece-ac87-d919cc97d035_Torque,b8d46933-5220-4bad-8221-c54715a835fd_Ang__deg__,b8d46933-5220-4bad-8221-c54715a835fd_Torque,e6b8d50e-a042-4a6a-8d75-6519b3c02934_Ang__deg__,e6b8d50e-a042-4a6a-8d75-6519b3c02934_Torque,f269a193-9198-434b-bc4c-fb46f7cdb1f3_Ang__deg__,f269a193-9198-434b-bc4c-fb46f7cdb1f3_Torque,4ad3b2c5-5198-4c11-ac65-86aef08f3db3_Torque,4ad3b2c5-5198-4c11-ac65-86aef08f3db3_Ang__deg__,1f100034-a532-4a58-8b6c-f4444e074afe_Torque,1f100034-a532-4a58-8b6c-f4444e074afe_Ang__deg__
12639700098,0472120354800150,2021-03-16 04:15:36.130,List(L14_Core_02),2021-03-16,null,null,null,null,16,13.69,null,null,null,null,null,null,null,null
12639700098,0472120502500153,2021-03-16 04:14:03.223,List(L14_Core_02),2021-03-16,null,null,null,null,11,13.69,null,null,null,null,null,null,null,null
12639700098,0472120502500153,2021-03-16 04:17:13.400,List(L14_Core_03),2021-03-16,null,null,90,154.71,null,null,null,null,null,null,null,null,null,null
12639700098,0482106351600004,2021-03-16 04:15:22.973,List(L14_Final_03),2021-03-16,null,null,null,null,null,null,34,10.45,null,null,null,null,null,null
12639700098,0482106351600004,2021-03-16 04:15:55.177,List(L14_Final_03),2021-03-16,null,null,null,null,null,null,49,10.48,null,null,null,null,null,null
12639700098,0482106351600004,2021-03-16 04:17:06.337,List(L14_Final_04),2021-03-16,null,null,null,null,null,null,null,null,988,17.04,null,null,null,null
12639700098,0482106351600004,2021-03-16 04:17:26.523,List(L14_Final_04),2021-03-16,56,13.69,null,null,null,null,null,null,null,null,null,null,null,null
12639700098,0482106351600004,2021-03-16 04:17:31.570,List(L14_Final_04),2021-03-16,62,13.64,null,null,null,null,null,null,null,null,null,null,null,null
12639700098,0482106351600004,2021-03-16 04:17:37.837,List(L14_Final_04),2021-03-16,80,13.73,null,null,null,null,null,null,null,null,null,null,null,null
12639700098,0482106351600004,2021-03-16 04:17:45.057,List(L14_Final_04),2021-03-16,69,13.75,null,null,null,null,null,null,null,null,null,null,null,null


In [0]:
%sql
drop table if exists pivot_results_table;
create table pivot_results_table
using DELTA
location 'dbfs:/acerta/output/cube/pivot/Borg_Warner_XYZ_L14/';

OPTIMIZE pivot_results_table ZORDER BY (serial_number);

path,metrics
dbfs:/acerta/output/cube/pivot/Borg_Warner_XYZ_L14,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 12, List(minCubeSize(107374182400), List(0, 0), List(12, 79774), 0, List(0, 0), 0, null), 0, 12, 12, false)"


In [0]:
%sql describe history pivot_results_table;

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata
1,2021-11-22T15:21:02.000+0000,1094680775067414,ljin@acerta.ai,MERGE,"Map(predicate -> (((s.`part_number` = u.`part_number`) AND (s.`serial_number` = u.`serial_number`)) AND ((s.`timestamp` = u.`timestamp`) AND (s.`station_list` = u.`station_list`))), matchedPredicates -> [{""actionType"":""update""}], notMatchedPredicates -> [{""actionType"":""insert""}])",null,List(1689534426296516),1110-132154-x22gtc30,0,WriteSerializable,false,"Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 12, executionTimeMs -> 2665, numTargetRowsInserted -> 297, scanTimeMs -> 363, numTargetRowsUpdated -> 0, numOutputRows -> 297, numTargetChangeFilesAdded -> 0, numSourceRows -> 297, numTargetFilesRemoved -> 0, rewriteTimeMs -> 2279)",null
0,2021-11-22T15:20:51.000+0000,1094680775067414,ljin@acerta.ai,WRITE,"Map(mode -> Append, partitionBy -> [""measurement_date""])",null,List(1689534426296516),1110-132154-x22gtc30,null,SnapshotIsolation,true,"Map(numFiles -> 0, numOutputBytes -> 0, numOutputRows -> 0)",null


In [0]:
### This is the seconda phase of transformation which compute 'run' information into the dataframe.
### this is used by RCA analysis.

from pyspark.sql.functions import col, from_json, to_json, concat, lit, window, when, desc, concat_ws, regexp_replace, collect_set
import pyspark.sql.functions as f
from pyspark.sql.window import Window


#**********************************************************************************************************************
# LOAD PIVOT TABLE
#**********************************************************************************************************************
pivot_load_df = spark.read.format('delta').load(pivotRootDir + "Borg_Warner_XYZ_L14/")


#**********************************************************************************************************************
# READ FEATURES LIST FROM SCHEMA TABLE
#**********************************************************************************************************************
dataSourceId, schemaVersion = '0.1.0', '0.1.1'
pivot_feature_cols = getFeatureListLocal(dataSourceId, schemaVersion)


#**********************************************************************************************************************
# DEFINE WINDOWS FOR AGGREGATION
#**********************************************************************************************************************

w1 = Window.partitionBy(["part_number", "serial_number"]).orderBy("timestamp")
w2 = Window.partitionBy(["part_number", "serial_number", "station"]).orderBy("timestamp")
w3 = Window.partitionBy(["part_number", "serial_number", "station", "consecutive_idx"]).orderBy(f.desc("timestamp"))
w4 = Window.partitionBy(["part_number", "serial_number"]).rowsBetween(Window.unboundedPreceding,Window.unboundedFollowing)
w5 = Window.partitionBy(["part_number", "serial_number", "target_operation"]).orderBy("target_test_date")


#**********************************************************************************************************************
# TRANSFORM STEP: add derived columns to prepare for path extraction
#
# New column details:
# 1. op_order: order of operations for a part goes through the assembly line, starting from 1 for the first operation
# 2. op_rerun: # of times a part passes a station
# 3. consecutive_idx (TEMPORARY): consecutive visits of a part to the same station (i.e. no other station in between) have the same consecutive_idx
# 4. is_last_measure: if a part is consecutively measured at a station, only the last measurement have is_last_measure = 1
#**********************************************************************************************************************

run_order_df = (pivot_load_df
                .withColumn("station", f.element_at("station_list", 1))
                .withColumn("op_order", f.dense_rank().over(w1))
                .withColumn("op_rerun", f.dense_rank().over(w2))
                .withColumn("consecutive_idx", col("op_order")-col("op_rerun"))
                .withColumn("is_last_measure", f.dense_rank().over(w3))
                .select("part_number", "serial_number", "station", "timestamp", "op_order", "op_rerun", "is_last_measure", *pivot_feature_cols)
               )


#**********************************************************************************************************************
# TRANSFORM STEP: identify 'path'(s) for each unit
#
# OUTPUT:  a dataframe with one row for each 'path', identified by 
#          1) the part and serial number of the assembled unit (part_number, serial_number)
#          2) the target station and the corresponding test date (target_operation, target_test_date)
#          3) a pass (1)/fail (0) flag indicating whether the part passed or failed at the target station (pass_fail)
#          4) a run number indicating the number of attempts for the unit to reach the target station
#**********************************************************************************************************************

target_ops_df = (run_order_df.withColumn("max_op_order", f.max("op_order").over(w4))
                      .filter((col("station")==target) & (col("is_last_measure")==1))
                      .withColumn("max_target_rerun", f.max("op_rerun").over(w4))
                      .withColumn("pass_fail", f.when(col("op_order")==col("max_op_order"), 2)
                                                 .when(col("op_rerun")==col("max_target_rerun"), 1)
                                                 .when(col("op_rerun")<col("max_target_rerun"), 0)
                                                 .otherwise(-1)
                                 )
                      .select("part_number", "serial_number", "station", "timestamp", "pass_fail")
                      .withColumnRenamed("timestamp", "target_test_date")
                      .withColumnRenamed("station", "target_operation")
                      .withColumn("run_number", f.row_number().over(w5))
                     )


#**********************************************************************************************************************
# TRANSFORM STEP: associate the pivotted sensor measurements to their corresponding path
#**********************************************************************************************************************

target_ops_df.registerTempTable("target_ops")
run_order_df.registerTempTable("run_order")
join_target_with_ops_sql = f"""
select o.part_number as part_number, o.serial_number as serial_number, run_number, 
    pass_fail, station, timestamp, `{'`,`'.join(pivot_feature_cols)}`,
	dense_rank() over (
		partition by o.part_number, o.serial_number, target_test_date, station order by timestamp
	) as op_rerun_in_path
from target_ops o join run_order r 
on o.part_number=r.part_number and o.serial_number=r.serial_number and target_test_date>=timestamp 
"""
cube_run_df= spark.sql(join_target_with_ops_sql)



#**********************************************************************************************************************
# WRITE CUBE TO DISK
# Write cube dataframe to parquet on disk. This completes the cube transformation.
#**********************************************************************************************************************

saveCube = cube_run_df.write.mode('append').parquet(cubeRunRootDir + "Borg_Warner_XYZ_L14/")  

display(cube_run_df)

part_number,serial_number,run_number,pass_fail,station,timestamp,b8d46933-5220-4bad-8221-c54715a835fd_Torque,30963d81-db2e-467f-8a4c-28a37a15da8c_Ang__deg__,e6b8d50e-a042-4a6a-8d75-6519b3c02934_Torque,4ad3b2c5-5198-4c11-ac65-86aef08f3db3_Torque,a65c94ea-4616-4ece-ac87-d919cc97d035_Ang__deg__,e6b8d50e-a042-4a6a-8d75-6519b3c02934_Ang__deg__,30963d81-db2e-467f-8a4c-28a37a15da8c_Torque,1f100034-a532-4a58-8b6c-f4444e074afe_Torque,1f100034-a532-4a58-8b6c-f4444e074afe_Ang__deg__,a65c94ea-4616-4ece-ac87-d919cc97d035_Torque,b8d46933-5220-4bad-8221-c54715a835fd_Ang__deg__,f269a193-9198-434b-bc4c-fb46f7cdb1f3_Ang__deg__,4ad3b2c5-5198-4c11-ac65-86aef08f3db3_Ang__deg__,f269a193-9198-434b-bc4c-fb46f7cdb1f3_Torque,op_rerun_in_path
12639700098,0482106351600004,1,2,L14_Core_03,2021-03-16 04:08:46.070,null,null,null,null,90,null,null,null,null,143.05,null,null,null,null,1
12639700098,0482106351600004,1,2,L14_Final_03,2021-03-16 04:15:22.973,null,null,10.45,null,null,34,null,null,null,null,null,null,null,null,1
12639700098,0482106351600004,1,2,L14_Final_03,2021-03-16 04:15:55.177,null,null,10.48,null,null,49,null,null,null,null,null,null,null,null,2
12639700098,0482106351600004,1,2,L14_Final_04,2021-03-16 04:17:06.337,null,null,null,null,null,null,null,null,null,null,null,988,null,17.04,1
12639700098,0482106351600004,1,2,L14_Final_04,2021-03-16 04:17:26.523,null,56,null,null,null,null,13.69,null,null,null,null,null,null,null,2
12639700098,0482106351600004,1,2,L14_Final_04,2021-03-16 04:17:31.570,null,62,null,null,null,null,13.64,null,null,null,null,null,null,null,3
12639700098,0482106351600004,1,2,L14_Final_04,2021-03-16 04:17:37.837,null,80,null,null,null,null,13.73,null,null,null,null,null,null,null,4
12639700098,0482106351600004,1,2,L14_Final_04,2021-03-16 04:17:45.057,null,69,null,null,null,null,13.75,null,null,null,null,null,null,null,5
12639700098,0482106452300007,1,2,L14_Final_04,2021-03-16 04:10:50.377,null,null,null,null,null,null,null,null,null,null,null,992,null,17.08,1
12639700098,0482106452300007,1,2,L14_Final_04,2021-03-16 04:11:11.290,null,55,null,null,null,null,13.78,null,null,null,null,null,null,null,2
